# Game 3 - Hospital Resources

Skillfully allocating medical resources during a pandemic can save lives. Unexpected and extreme medical events not only disrupt hospitals' normal routines and protocol, but strain resources such as trained staff, space and pharmaceuticals, in addition to PPE and life-saving medical equipment. Determining how to most effectively allocate these resources is paramount to assuring the wellbeing of Sailors far from port.

In this scenario, you will oversee the medical bay of the USS GHOST where COVID-19 has taken hold of the crew. Your task is to create an algorithm to deploy the ship's available resources in a manner that will ensure as many Sailors as possible recover from the virus.

## brainstorming:
- reinforcement learning?

In [255]:
import pandas as pd
import numpy as np
from itertools import combinations

pd.set_option("display.max_rows", 100)

* +30 10 ventilators (reusable)
* +20 10 oxygen masks (reusable)
    * cannot be combined with ventilator
* +15 10 plasma
* +30 7 remdesivir
* +25 20 Dexamethasone
* +15 10 casirivimab
* +10 17 chloroquine
* 30 beds
    * bed boosts health 5 points
    * bed allocation required for other treatments
* final health score = final health + bed bonus + rate of decline + treatment bonuses
* penalty of -100 if any sailor final health is 0


In [142]:
sailors_df = pd.read_csv("day1data.csv", header=1, index_col=0)
sailors_df2 = pd.read_csv("day2data.csv", header=1, index_col=0)
# sailors_df.drop("Unnamed: 3", axis=1, inplace=True)
resource_list = ["Bed", "Ventilator", "Remdesivir", "Dexamethasone", "Plasma", "Casirivimab", "Supplemental oxygen", "Chloroquine"]
for resource in resource_list:
    sailors_df[resource] = 0
    sailors_df2[resource] = 0
sailors_df["Used"] = ""
sailors_df2["Used"] = ""
columns = ["resource", "count", "amt_used", "reusable", "health_bonus"]
data = np.array([["Bed", 30, 0, True, 5],
          ["Ventilator", 10, 0, True, 30],
          ["Supplemental oxygen", 10, 0, True, 20],
          ["Plasma", 10, 0, False, 15],
          ["Remdesivir", 7, 0, False, 30],
          ["Dexamethasone", 20, 0, False, 25],
          ["Casirivimab", 10, 0, False, 15],
          ["Chloroquine", 17, 0, False, 10]])
resources_df = pd.DataFrame(data=data, columns=columns)
for col in ["count", "amt_used", "health_bonus"]:
    resources_df[col] = pd.to_numeric(resources_df[col])
print(resources_df,"\n")

              resource  count  amt_used reusable  health_bonus
0                  Bed     30         0     True             5
1           Ventilator     10         0     True            30
2  Supplemental oxygen     10         0     True            20
3               Plasma     10         0    False            15
4           Remdesivir      7         0    False            30
5        Dexamethasone     20         0    False            25
6          Casirivimab     10         0    False            15
7          Chloroquine     17         0    False            10 



In [249]:
import random
sailors_df3 = pd.DataFrame()
for i in range(80):
    sailors_df3.at[i, "Health"] = random.randrange(1, 101)
    sailors_df3.at[i, "Rate of health decline"] = random.randrange(-99, 0)
resource_list = ["Bed", "Ventilator", "Remdesivir", "Dexamethasone", "Plasma", "Casirivimab", "Supplemental oxygen", "Chloroquine"]
for resource in resource_list:
    sailors_df3[resource] = 0
sailors_df3["Used"] = ""
sailors_df["Health"].mean(), sailors_df2["Health"].mean(), sailors_df3["Health"].mean()

(69.7, 63.77777777777778, 44.5)

In [230]:
sailors_df3_temp = sailors_df3.copy()
sailors_df3_temp["Total"] = sailors_df3["Health"]+sailors_df3["Rate of health decline"]
sailors_df3_temp.sort_values(by=["Total"], inplace=True)
sailors_df3_temp.head(30)

,Health,Rate of health decline,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Used,Total
18,4.0,-74.0,0,0,0,0,0,0,0,0,,-70.0
36,21.0,-89.0,0,0,0,0,0,0,0,0,,-68.0
26,21.0,-86.0,0,0,0,0,0,0,0,0,,-65.0
12,24.0,-86.0,0,0,0,0,0,0,0,0,,-62.0
31,17.0,-72.0,0,0,0,0,0,0,0,0,,-55.0
23,22.0,-74.0,0,0,0,0,0,0,0,0,,-52.0
33,46.0,-97.0,0,0,0,0,0,0,0,0,,-51.0
15,34.0,-84.0,0,0,0,0,0,0,0,0,,-50.0
48,36.0,-81.0,0,0,0,0,0,0,0,0,,-45.0
47,41.0,-85.0,0,0,0,0,0,0,0,0,,-44.0


In [250]:
def use_resource(sailors_df, resources_df, resource, indices):
    """
    resource = resource name as string
    indices = Pandas index list
    Ex. for single indices do [df.index[position]]
    Ex. for range of values do df.index[start:end]
    """
    bonus_dict = {}
    for r, bonus in zip(resources_df["resource"], resources_df["health_bonus"]):
        bonus_dict[r] = int(bonus)

    sailors_df.loc[indices, resource] = 1
    sailors_df.loc[indices, "Total"] += bonus_dict[resource]
    resources_df.loc[resources_df["resource"] == resource, "amt_used"] += len(indices)
    resources_df.loc[resources_df["resource"] == resource, "count"] -= len(indices)

def allocate_reusable(sailors_df, resources_df):    
    add_beds = sailors_df.index[:30]
    use_resource(sailors_df, resources_df, "Bed", add_beds)
    
    add_ventilators = sailors_df.index[:10]
    use_resource(sailors_df, resources_df, "Ventilator", add_ventilators)
    
    oxymask_candidates = sailors_df[(sailors_df["Ventilator"]==0) & (sailors_df["Bed"]>0)]
    add_oxymask = oxymask_candidates.index[:10]
    use_resource(sailors_df, resources_df, "Supplemental oxygen", add_oxymask)

def ded(sailors_df, resources_df):
    def get_bonus(tups):
        total = 0
        for name, bonus in tups:
            total += bonus
        return total
    def get_names(tups):
        names = []
        for name, bonus in tups:
            names.append(name)
        return names

    mindict = {}
    minlist = []
    dying = sailors_df.loc[(sailors_df["Total"] <= 0) & (sailors_df["Bed"] > 0)]
    print(dying)
    for sailor in dying.itertuples():
        nonreusable = resources_df[(resources_df["reusable"]=="False") & (resources_df["count"] > 0)]
        tups = list(zip(nonreusable["resource"], nonreusable["health_bonus"]))
        y = sailor.Total
        comb_length = 1
        print(sailor.Index, tups)
        while len(minlist) == 0 and comb_length < len(tups):
            for comb in combinations(tups, comb_length):
                x = y + get_bonus(comb)
                mindict[x] = get_names(comb)
            minlist = [k for k in mindict.keys() if k > 0]
            comb_length += 1
        if len(minlist) == 0:
#             print("Sailor with Health:", sailors_df.iloc[2]["Health"], "is dying.")
            continue
        m = min(minlist)
        for resource in mindict[m]:
            use_resource(sailors_df, resources_df, resource, [sailor.Index])
        sailors_df.loc[sailor.Index, "Used"] = ','.join(mindict[m])
        mindict = {}
        minlist = []

def boosting(sailors_df, resources_df, threshold):
    def get_bonus(tups):
        total = 0
        for name, bonus in tups:
            total += bonus
        return total
    def get_names(tups):
        names = []
        for name, bonus in tups:
            names.append(name)
        return names
    treatable = sailors_df.loc[(sailors_df["Bed"] > 0) & (sailors_df["Total"] > 0)]
    mindict = {}
    minlist = []
    for sailor in treatable.itertuples():
        unavailable = sailor.Used.split(',')
        available = resources_df[(~resources_df["resource"].isin(unavailable)) \
                                 & (resources_df["count"] > threshold) \
                                 & (resources_df["reusable"]=="False")]
        
        tups = list(zip(available["resource"], available["health_bonus"]))
        y = sailor.Total
        comb_length = len(tups)
        while len(minlist) == 0 and comb_length > 0:
            for comb in combinations(tups, comb_length):
                x = y + get_bonus(comb)
                mindict[x] = get_names(comb)
            minlist = [k for k in mindict.keys() if k < 101]
            comb_length -= 1
        if len(minlist) == 0:
            continue
        m = min(minlist)
        for resource in mindict[m]:
            
            use_resource(sailors_df, resources_df, resource, [sailor.Index])
        sailors_df.loc[sailor.Index, "Used"] = ','.join(mindict[m])
        mindict = {}
        minlist = []

def allocation(sailors_df, resources_df, threshold):
    resources_df = resources_df.copy()
    sailors_df = sailors_df.copy()
    sailors_df["Total"] = sailors_df["Health"]+sailors_df["Rate of health decline"]
    sailors_df.sort_values(by=["Total"], inplace=True)
    
    allocate_reusable(sailors_df, resources_df)
    sailors_df["After Reusable"] = sailors_df["Total"]
    ded(sailors_df, resources_df)
    boosting(sailors_df, resources_df, threshold)
    
    return sailors_df, resources_df

def calc_final_score(allocation_df, resources_df):
    final_health = allocation_df["Total"].apply(lambda x: 100 if x > 100 else x)
    final_health = final_health.apply(lambda x: -100 if x < 1 else x)
    resources_df = resources_df[resources_df["reusable"]=="False"]
    resource_bonus = (resources_df["count"]*resources_df["health_bonus"]).sum()/2
    return final_health.sum() + resource_bonus


In [256]:
a_df, r_df = allocation(sailors_df, resources_df, 30)
a_df2, r_df2 = allocation(sailors_df2, r_df, 30)
a_df3, r_df3 = allocation(sailors_df3, r_df2, 0)
print(r_df3)
a_df3

    Health  Rate of health decline  Bed  Ventilator  Remdesivir  \
ID                                                                
20      20                     -80    1           1           0   
15      39                     -89    1           1           0   
14      28                     -67    1           1           0   

    Dexamethasone  Plasma  Casirivimab  Supplemental oxygen  Chloroquine Used  \
ID                                                                              
20              0       0            0                    0            0        
15              0       0            0                    0            0        
14              0       0            0                    0            0        

    Total  After Reusable  
ID                         
20    -25             -25  
15    -15             -15  
14     -4              -4  
20 [('Plasma', 15), ('Remdesivir', 30), ('Dexamethasone', 25), ('Casirivimab', 15), ('Chloroquine', 10)]
15 [('Plasma'

26 [('Plasma', 15), ('Remdesivir', 30), ('Dexamethasone', 25), ('Casirivimab', 15), ('Chloroquine', 10)]
44 [('Plasma', 15), ('Remdesivir', 30), ('Dexamethasone', 25), ('Casirivimab', 15), ('Chloroquine', 10)]
9 [('Plasma', 15), ('Remdesivir', 30), ('Dexamethasone', 25), ('Casirivimab', 15), ('Chloroquine', 10)]
40 [('Plasma', 15), ('Remdesivir', 30), ('Dexamethasone', 25), ('Casirivimab', 15), ('Chloroquine', 10)]
54 [('Plasma', 15), ('Remdesivir', 30), ('Dexamethasone', 25), ('Casirivimab', 15), ('Chloroquine', 10)]
39 [('Plasma', 15), ('Remdesivir', 30), ('Dexamethasone', 25), ('Casirivimab', 15), ('Chloroquine', 10)]
76 [('Plasma', 15), ('Remdesivir', 30), ('Dexamethasone', 25), ('Casirivimab', 15), ('Chloroquine', 10)]
28 [('Plasma', 15), ('Remdesivir', 30), ('Dexamethasone', 25), ('Casirivimab', 15), ('Chloroquine', 10)]
22 [('Plasma', 15), ('Remdesivir', 30), ('Dexamethasone', 25), ('Casirivimab', 15), ('Chloroquine', 10)]
66 [('Plasma', 15), ('Remdesivir', 30), ('Dexamethasone'

,Health,Rate of health decline,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Used,Total,After Reusable
14,3.0,-95.0,1,1,1,0,1,1,0,1,Chloroquine,13.0,-57.0
47,3.0,-92.0,1,1,1,1,1,0,0,1,"Plasma,Chloroquine",26.0,-54.0
67,7.0,-81.0,1,1,0,1,1,1,0,1,"Plasma,Chloroquine",26.0,-39.0
37,24.0,-97.0,1,1,0,1,1,1,0,1,"Plasma,Chloroquine",27.0,-38.0
19,8.0,-72.0,1,1,1,0,1,0,0,1,"Plasma,Chloroquine",26.0,-29.0
12,15.0,-74.0,1,1,0,1,1,0,0,1,"Plasma,Chloroquine",26.0,-24.0
20,26.0,-84.0,1,1,0,1,1,0,0,1,"Plasma,Chloroquine",27.0,-23.0
6,14.0,-67.0,1,1,0,1,1,0,0,1,"Plasma,Chloroquine",32.0,-18.0
65,43.0,-93.0,1,1,0,1,1,0,0,1,"Plasma,Chloroquine",35.0,-15.0
25,26.0,-76.0,1,1,0,1,0,0,0,1,Chloroquine,20.0,-15.0


In [114]:
a_df.drop(["Used", "After Reusable"], axis=1).sort_values(by="ID").to_csv("Challenge3_Day1Submission.csv")
a_df2.drop(["Unnamed: 3", "Used", "After Reusable"], axis=1).sort_values(by="ID").to_csv("Challenge3_Day2Submission.csv")

30.0

In [ ]:
min_count = 5
for each sailor with Total above 0 with a bed:
    get list of resources that have not been applied to sailor and are above min_count
    get max combination of resources that can be applied to sailor that will not make Total > 100
        start with most resources and move backwards
    apply resources
    reset